In [44]:
import pandas as pd
df = pd.read_csv(r'C:\Users\adamb\Desktop\Base_2017_1.csv', encoding='utf8', sep=';')

In [45]:
df.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,1,BOTICÁRIO,1,CABELOS,27/03/2018,9
1,4,VULT,1,CABELOS,03/02/2018,3
2,2,EUDORA,1,CABELOS,21/10/2018,7
3,2,EUDORA,4,SOLAR,22/04/2018,13
4,5,BELEZA NA WEB,3,MAQUIAGEM,21/03/2018,9


In [8]:
df.shape

(1000, 6)

In [9]:
df.dtypes

ID_MARCA       int64
MARCA         object
ID_LINHA       int64
LINHA         object
DATA_VENDA    object
QTD_VENDA      int64
dtype: object

In [10]:
df.describe()

,ID_MARCA,ID_LINHA,QTD_VENDA
count,1000.00000,1000.000000,1000.000000
mean,2.99600,2.997000,10.356000
std,1.43668,1.421623,5.653249
min,1.00000,1.000000,1.000000
25%,2.00000,2.000000,5.000000
50%,3.00000,3.000000,10.000000
75%,4.00000,4.000000,15.000000
max,5.00000,5.000000,20.000000


In [12]:
df.columns

Index(['ID_MARCA', 'MARCA', 'ID_LINHA', 'LINHA', 'DATA_VENDA', 'QTD_VENDA'], dtype='object')

In [19]:
df.isnull().any()

ID_MARCA      False
MARCA         False
ID_LINHA      False
LINHA         False
DATA_VENDA    False
QTD_VENDA     False
dtype: bool

In [20]:
df.notnull().all()

ID_MARCA      True
MARCA         True
ID_LINHA      True
LINHA         True
DATA_VENDA    True
QTD_VENDA     True
dtype: bool

In [23]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [27]:
df.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,1,BOTICÁRIO,1,CABELOS,27/03/2018,9
1,4,VULT,1,CABELOS,03/02/2018,3
2,2,EUDORA,1,CABELOS,21/10/2018,7
3,2,EUDORA,4,SOLAR,22/04/2018,13
4,5,BELEZA NA WEB,3,MAQUIAGEM,21/03/2018,9


In [144]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import exc
#import table_columns
from io import StringIO
import re
import sys

In [145]:
pd.options.display.max_columns=50

In [146]:
engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    'postgres',
    'Stage2!',
    'stage.chlnyxxxxxyy.us-east-2.rds.amazonaws.com',
    '5432' ,
    'postgres'
))

In [147]:
%env SCHEMA = bot_stage

env: SCHEMA=bot_stage


In [148]:
path = 'C:\\Users\\adamb\\Desktop\\BOT\\'
lista_arquivos = os.listdir(path)

In [149]:
dici_arquivos = {
    'sales': 'Base_{}.csv'
}

In [150]:
def have_tables(dici_arquivos):
    tabelas_faltante = []
    for tabela in dici_arquivos:
        if dici_arquivos[tabela] == '':
            tabelas_faltante.append(tabela)
    
    if len(tabelas_faltante)>0:
        print('Estão faltando as seguintes tabelas:\n',tabelas_faltante)

In [151]:
def get_list_of_dataframe(arquivo):
    prefixo_arquivo = arquivo.replace('{}.csv', '')
    arquivo_diretorio = os.listdir(path)
    lista_arquivos_para_agrupar = [arquivo for arquivo in arquivo_diretorio if arquivo.startswith(prefixo_arquivo) ]
    # Agrupar arquivos em 1 único dataframe
    lista_df = []
    print('lido os arquivos: ')
    for arquivo in lista_arquivos_para_agrupar:  
        print(arquivo)
        lista_df.append(text_file_to_dataframe(path+arquivo))
    return lista_df

In [152]:
def text_file_to_dataframe(path_file):
    try:
        return pd.read_csv(path_file, sep=';', encoding='utf8', dtype='str')
    except pd.errors.ParserError as e:
        with open("temp.log", "w") as f: 
            original_stderr = sys.stderr
            sys.stderr = f
            df = pd.read_csv(path_file, sep=';', encoding='utf8', error_bad_lines=False, dtype='str')
            sys.stderr = original_stderr
            
        # Exibir as linhas que apresentaram erro    
        with open("temp.log", "r") as f: 
            lines = f.readlines()[0].split('\\n')
            errors = []
            for line in lines:
                errors += [re.findall('\d+', line)]
                errors_lines = [int(first[0]) for first in errors if len(first)>0]
                
        print('Arquivos com linhas quebradas: {} \nLinhas:{}'.format(path_file, errors_lines))
        
        
    return df

In [153]:
def read_file(tabela, path):
    arquivo = dici_arquivos[tabela]
    # verificar se a tabela está dividida em multiplos arquivo
    if '{}' in arquivo:
        lista_df = get_list_of_dataframe(arquivo)
        df = pd.concat(lista_df, sort=False)
    else:
        df = text_file_to_dataframe(path+arquivo)
        
    df.archive = arquivo
    df.table = tabela
    return df

In [154]:
'''
Passar essa responsabilidade para o read_file
'''
def read_from_csv_deleting_lines(tabela, lines=[]):
    with open(path + dici_arquivos[tabela], "r", encoding='utf8') as in_file:
        arquivo_lista = in_file.read().splitlines()
        for line in lines:
            if str(line)[0]=='>':
                line = int(line[1:])
                del arquivo_lista[line-1:]
            else: 
                del arquivo_lista[line-1]
    arquivo_texto = StringIO('\n'.join(arquivo_lista))
    df = pd.read_csv(arquivo_texto, sep=';')
    return df

In [155]:
def load_to_db(df):
    df.to_sql('sales'.format(tabela),
          con=engine,
          schema=os.getenv('SCHEMA'),
          if_exists='replace',
          index=False,
          chunksize=10000,
          method='multi'
         )

In [156]:
have_tables(dici_arquivos)

In [157]:
# Tabela Sales

In [158]:
#tabela = 'sales'
#colunas = table_columns.sales

In [159]:
df = read_file(tabela, path)

lido os arquivos: 
Base_2017_1.csv
Base_2018_2.csv
Base_2019_3.csv


In [160]:
df.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,1,BOTICÁRIO,1,CABELOS,27/03/2018,9
1,4,VULT,1,CABELOS,03/02/2018,3
2,2,EUDORA,1,CABELOS,21/10/2018,7
3,2,EUDORA,4,SOLAR,22/04/2018,13
4,5,BELEZA NA WEB,3,MAQUIAGEM,21/03/2018,9


In [162]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 3000, dtype: bool

In [163]:
df.isnull().any()

ID_MARCA      False
MARCA         False
ID_LINHA      False
LINHA         False
DATA_VENDA    False
QTD_VENDA     False
dtype: bool

In [164]:
df.shape

(3000, 6)

In [161]:
load_to_db(df)